# Demonstration of Masks and Filters
You'll learn how to exploit intensity patterns to select sub-regions of an array, and you'll use convolutional filters to detect interesting features. 

This demo is a jupyter notebook, i.e. intended to be run step by step.

Author: Eric Einspänner
<br>
Contributor: Nastaran Takmilhomayouni

First version: 6th of July 2023


Copyright 2023 Clinic of Neuroradiology, Magdeburg, Germany

License: Apache-2.0

## Table of contents
0. [Initial Set-Up for Google Colab](#initial-set-up-for-google-colab)
1. [Initial Set-Up (offline)](#initial-set-up-offline)
2. [Load images](#Load-Images)
3. [Intensity](#Intensity)
4. [Histograms](#Histograms)
5. [Create a mask](#Create-a-Mask)
6. [Tune a mask](#Tune-a-Mask)
    - [Exercise (Tune mask)](#exercise-tune-mask)
7. [Filter convolutions](#Filter-convolutions)
    - [Exercise (Filtering)](#exercise-filtering)
8. [Smoothing](#Smoothing)
8. [Detect edges](#Detect-Edges)
    - [Exercise](#Exercise-2)

## Initial Set-Up for Google Colab
<u> Execute these code blocks just in Google Colab! </u>

In [ ]:
!git clone https://github.com/University-Clinic-of-Neuroradiology/python-bootcamp.git

In [ ]:
import os
import sys
from google.colab import output
output.enable_custom_widget_manager()

sys.path.insert(0,'/content/python-bootcamp/notebooks/ImageAnalysis')
os.chdir(sys.path[0])

In [ ]:
%pip install -q ipympl numpy matplotlib imageio pydicom SciPy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import imageio
import scipy.ndimage as ndi
import pydicom
from pydicom.data import get_testdata_file

## Initial Set-Up (offline)

In [ ]:
# Make sure figures appears inline and animations works
# Edit this to ""%matplotlib notebook" when using the "classic" jupyter notebook interface
%matplotlib widget

In [ ]:
# Initial imports etc
import numpy as np
import matplotlib.pyplot as plt

import imageio
import scipy.ndimage as ndi
import pydicom
from pydicom.data import get_testdata_file

## --- Start notebook ---

## Load Images

In [ ]:
#  now we load test data files (CT and MR)
ct_file = get_testdata_file("CT_small.dcm")

# Load the CT file (image)
im = imageio.imread(ct_file)

## Intensity
X-ray absorption is highest in dense tissue such as bone, so the resulting intensities should be high.

The image datatype determines the range of possible intensities: e.g., 8-bit unsigned integers (`uint8`) can take values in the range of 0 to 255 or 16-bit signed integers (`int16`) with a range of -32768 to 32767. A colorbar can be helpful for connecting these values to the visualized image.

In [ ]:
# Print the image's data type, minimum and maximum intensity
print('Data type:', im.dtype)
print('Min. value:', im.min())
print('Max value:', im.max())

You can see, that we have an `int16` datatype with a wide range of intensity values. For better representation and uniform values, we perform min-max normalisation:

In [ ]:
# min-max normalisation
im_old = im                                     # save original image
im = (im - im.min()) / (im.max() - im.min())    # normalise to 0-1 range

# Print the image's data type, minimum and maximum intensity
print('Data type:', im.dtype)
print('Min. value:', im.min())
print('Max value:', im.max())

The following function `format_and_render_plot()` is just a simplify formatting method for the plots in this notebook. In the next cells we will modify the the function a little bit. Try to understand what we changed and why. Change the arguments and explore what happens!

In [ ]:
def format_and_render_plot(axis=False, legend=False):
    '''
    Custom function to simplify common formatting operations for exercises. Operations include: 
    1. Turning off axis grids and legends, if not explicitly requested.
    2. Calling `plt.tight_layout` to improve subplot spacing.
    3. Calling `plt.show()` to render plot.
    '''
    fig = plt.gcf()
    for ax in fig.axes:
        if not axis:
            ax.axis('off')
        if legend:  
            ax.legend(loc='center right')  
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot the grayscale images
fig, axes = plt.subplots(1, 2, sharex=True)
axes[0].imshow(im_old, cmap='gray')
axes[0].set_title('without normalization', fontweight ="bold")
axes[1].imshow(im, cmap='gray')
axes[1].set_title('with normalization', fontweight ="bold")
format_and_render_plot()

## Histograms
Histograms display the distribution of values in your image by binning each element by its intensity then measuring the size of each bin.

The area under a histogram is called the cumulative distribution function. It measures the frequency with which a given range of pixel intensities occurs.

In [ ]:
# Create a 256-bin histogram, binned at each possible value
hist = ndi.histogram(im, min=im.min(), max=im.max(), bins=256)

# Create a cumulative distribution function
cdf = hist.cumsum() / hist.sum()

# Plot the histogram and CDF
fig, axes = plt.subplots(2, 1, sharex=True)         # sharex=True shares the x-axis between the top and bottom subplot
axes[0].plot(hist, label='Histogram')               # label='Histogram' labels this line as "Histogram" for the legend
axes[1].plot(cdf, label='CDF')                      # label='CDF' labels this line as "CDF" for the legend
format_and_render_plot(axis=True, legend=True)      # axis=True turns on axis grids for the plot; legend=True turns on the legend

## Create a Mask
Masks are the primary method for removing or selecting specific parts of an image. They are binary arrays that indicate whether a value should be included in an analysis. Typically, masks are created by applying one or more logical operations to an image.

In this exercise, you use a simple intensity threshold to differentiate between soft tissue and bone.

<ins>Exercise:</ins> Try different intervals/thresholds by yourself!

In [ ]:
# Create soft tissue (st) and bone masks
# Try different intervals by yourself
mask_st = (im >= 0.2) & (im < 0.52)     # soft tissue mask
mask_bone = im >= 0.53                  # bone mask

# Plot the skin (0) and bone (1) masks
fig, axes = plt.subplots(1,2)
axes[0].imshow(mask_st, cmap='gray')
axes[0].set_title('soft tissue map', fontweight ="bold")
axes[1].imshow(mask_bone, cmap='gray')
axes[1].set_title('bone map', fontweight ="bold")
format_and_render_plot()

## Apply a Mask
Although masks are binary, they can be applied to images to filter out pixels where the mask is `False`.

NumPy's `where()` function is a flexible way of applying masks. It takes three arguments:

    np.where(condition, x, y)

`condition`, `x` and `y` can be either arrays or single values. This allows you to pass through original image values while setting masked values to 0.

In [ ]:
# Screen out non-bone pixels from "im"
im_bone = np.where(mask_bone, im, 0)

# Get the histogram of bone intensities
hist = ndi.histogram(im_bone, min=im_bone.min()+0.01, max=im_bone.max(), bins=255)

# Plot masked image and histogram
fig, axes = plt.subplots(2,1)
axes[0].imshow(im_bone, cmap='gray', label='Bone pixels')
axes[0].axis('off')
axes[1].plot(hist, label='Histogram')
format_and_render_plot(axis=True, legend=True)

## Tune a Mask
Imperfect masks can be tuned through the addition and subtraction of pixels. SciPy includes several useful methods for accomplishing these ends. These include:

- `binary_dilation`: Add pixels along edges
- `binary_erosion`: Remove pixels along edges
- `binary_opening`: Erode then dilate, "opening" areas near edges
- `binary_closing`: Dilate then erode, "filling in" holes

In [ ]:
# Create and tune bone mask
mask_dilate = ndi.binary_dilation(mask_bone, iterations=5)
mask_closed = ndi.binary_closing(mask_bone, iterations=5)

# Plot masked images
fig, axes = plt.subplots(1,3)
axes[0].imshow(mask_bone)
axes[0].set_title('original', fontweight ="bold")
axes[1].imshow(mask_dilate)
axes[1].set_title('dilated', fontweight ="bold")
axes[2].imshow(mask_closed)
axes[2].set_title('closed', fontweight ="bold")
format_and_render_plot()

### Exercise (Tune mask)
Try some other paramters and observe the changes:

In [ ]:
# Write your code here






In [ ]:
### Solution - just an example, try your own
# Let's create a funny mask ;)
mask_erosion = ndi.binary_erosion(mask_bone, iterations=5)
mask_opening = ndi.binary_opening(mask_bone, iterations=5)

# Plot masked images
fig, axes = plt.subplots(1,3)
axes[0].imshow(mask_bone)
axes[0].set_title('original', fontweight ="bold")
axes[1].imshow(mask_erosion)
axes[1].set_title('erosion', fontweight ="bold")
axes[2].imshow(mask_opening)
axes[2].set_title('opening', fontweight ="bold")
format_and_render_plot()

## Filter Convolutions
Filters are an essential tool in image processing. They allow you to transform images based on intensity values surrounding a pixel, rather than globally.

This step is also an important part in the most common DeepLearning models, calles convolutional layer.

In [ ]:
# Set filter weights
weights = [[0.11, 0.11, 0.11],
           [0.11, 0.11, 0.11], 
           [0.11, 0.11, 0.11]]

# Convolve the image with the filter
im_filt = ndi.convolve(im, weights)

# Plot the images
fig, axes = plt.subplots(1,2)
axes[0].imshow(im, cmap='gray')
axes[0].set_title('original', fontweight ="bold")
axes[1].imshow(im_filt, cmap='gray')
axes[1].set_title('with filter', fontweight ="bold")
format_and_render_plot()

### Exercise (Filtering)
Try some new filters and observe the changes.

<ins>Optional:</ins> Try to apply a sharpening filter! 

In [ ]:
# Write your code here






In [ ]:
### Solution
# Set filter weights for sharpening the image
weights = [[-1, -1, -1],
           [-1, 9, -1],
           [-1, -1, -1]]

# Convolve the image with the filter
im_filt = ndi.convolve(im, weights)

# Plot the images
fig, axes = plt.subplots(1,2)
axes[0].imshow(im, cmap='gray')
axes[0].set_title('original', fontweight ="bold")
axes[1].imshow(im_filt, cmap='gray')
axes[1].set_title('with filter', fontweight ="bold")
format_and_render_plot()

## Smoothing
Smoothing can improve the signal-to-noise ratio of your image by blurring out small variations in intensity. The Gaussian filter is excellent for this: it is a circular (or spherical) smoothing kernel that weights nearby pixels higher than distant ones.

The width of the distribution is controlled by the sigma argument, with higher values leading to larger smoothing effects.

<ins>Exercise:</ins> Try different sigmas by yourself!

In [ ]:
# Smooth "im" with Gaussian filters
# Try different sigmas  by yourself
im_s1 = ndi.gaussian_filter(im, sigma=1)
im_s3 = ndi.gaussian_filter(im, sigma=3)

# Draw bone masks of each image
fig, axes = plt.subplots(1,3)
axes[0].imshow(im, cmap='gray')
axes[0].set_title('original', fontweight ="bold")
axes[1].imshow(im_s1, cmap='gray')
axes[1].set_title('Gaussian (sigma=1)', fontweight ="bold")
axes[2].imshow(im_s3, cmap='gray')
axes[2].set_title('Gaussian (sigma=3)', fontweight ="bold")
format_and_render_plot()

## Detect Edges
Filters can also be used as "detectors." If a part of the image fits the weighting pattern, the returned value will be very high (or very low).

In the case of edge detection, that pattern is a change in intensity along a plane.

In [ ]:
# Set weights to detect vertical edges
weights = [[1, 0, -1],
           [1, 0, -1],
           [1, 0, -1]]

# Convolve "im" with filter weights
edges = ndi.convolve(im, weights)

# Draw the image in color
fig, axes = plt.subplots(1,2)
axes[0].imshow(im, cmap='gray')
axes[0].set_title('original', fontweight ="bold")
axes[1].imshow(edges, cmap='seismic', vmin=-0.8, vmax=0.8)
axes[1].set_title('edge filter', fontweight ="bold")
format_and_render_plot()

### Exercise 2
A filter detecting horizontal edges might look like this:

    weights = [[+1, +1, +1],
            [ 0,  0,  0],
            [-1, -1, -1]]


Apply the new filter to the image!

In [ ]:
# Write your code here (the solution is below)






In [ ]:
### Solution
# Set weights to detect vertical edges
weights = [[ 1,  1,  1], 
           [ 0,  0,  0],
           [-1, -1, -1]]

# Convolve "im" with filter weights
edges = ndi.convolve(im, weights)

# Draw the image in color
fig, axes = plt.subplots(1,2)
axes[0].imshow(im, cmap='gray')
axes[0].set_title('original', fontweight ="bold")
axes[1].imshow(edges, cmap='seismic', vmin=-0.8, vmax=0.8)
axes[1].set_title('Sobel filter', fontweight ="bold")
format_and_render_plot()